# ICLR 2025 OpenReview data collection

Uses OpenReview API v2 (api2.openreview.net). Same schema as ICLR 2024: submissions, official reviews, decisions, and full discussion threads (rebuttals, meta-reviews).

In [ ]:
# Install if needed (uncomment)
# !pip install openreview-py "urllib3<2.0"

import openreview
import pandas as pd
import tqdm
import time
import re
from pathlib import Path
from collections import defaultdict

OUTPUT_DIR = Path("ICLR/2025")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
RANDOM_SEED = 42

# Rate limit: OpenReview allows 60 requests/min. Delay between each forum request.
REQUEST_DELAY_SEC = 1.01
RATE_LIMIT_WAIT_SEC = 61  # wait when 429 (slightly > 60s reset window)
MAX_RETRIES = 10

In [ ]:
def get_all_descendants(root_id, reply_map):
    """Return all descendant notes under root_id (exclusive). reply_map: parent_id -> list of child notes."""
    descendants = []
    for child in reply_map.get(root_id, []):
        descendants.append(child)
        descendants.extend(get_all_descendants(child.id, reply_map))
    return descendants

In [ ]:
def _get_forum_notes_with_retry(client, paper_id):
    """Fetch forum notes with retry on 429 RateLimitError. Throttles to stay under 60 req/min."""
    for attempt in range(MAX_RETRIES):
        try:
            forum_notes = client.get_all_notes(forum=paper_id)
            return forum_notes
        except Exception as e:
            err_str = str(e)
            if "429" in err_str or "RateLimitError" in err_str or "Too many requests" in err_str:
                wait = RATE_LIMIT_WAIT_SEC
                match = re.search(r"try again in (\d+) seconds", err_str, re.I)
                if match:
                    wait = int(match.group(1)) + 5
                if attempt < MAX_RETRIES - 1:
                    tqdm.tqdm.write(f"Rate limited. Waiting {wait}s then retry ({attempt + 1}/{MAX_RETRIES})...")
                    time.sleep(wait)
                else:
                    raise
            else:
                raise
    return []


def get_iclr_2025_full_conversation():
    client = openreview.api.OpenReviewClient(baseurl="https://api2.openreview.net")
    venue_id = "ICLR.cc/2025/Conference"
    submission_invitation = f"{venue_id}/-/Submission"

    print(f"1. Getting submission list ({submission_invitation})...")
    submissions = client.get_all_notes(invitation=submission_invitation)
    print(f"2. Found {len(submissions)} submissions. Throttling ~{1/REQUEST_DELAY_SEC:.1f} req/s to avoid 429.")

    all_reviews_data = []

    for note in tqdm.tqdm(submissions):
        paper_id = note.id
        paper_title = note.content.get("title", {}).get("value")
        paper_number = note.number

        forum_notes = _get_forum_notes_with_retry(client, paper_id)
        time.sleep(REQUEST_DELAY_SEC)
        reply_map = defaultdict(list)
        reviews = []
        decision_note = None

        for n in forum_notes:
            if n.replyto:
                reply_map[n.replyto].append(n)
            if n.invitations and any("Decision" in inv for inv in n.invitations):
                decision_note = n
            if n.invitations and any("Official_Review" in inv for inv in n.invitations):
                reviews.append(n)

        decision_value = "None"
        if decision_note:
            decision_value = decision_note.content.get("decision", {}).get("value", "None")

        for review in reviews:
            review_id = review.id
            review_content = review.content
            discussion_nodes = get_all_descendants(review_id, reply_map)
            discussion_nodes.sort(key=lambda x: x.tmdate)

            transcript_lines = []
            for node in discussion_nodes:
                speaker = "Unknown"
                sigs = node.signatures
                if any("Authors" in s for s in sigs):
                    speaker = "Authors"
                elif any("Reviewer" in s for s in sigs):
                    speaker = sigs[0].split("/")[-1]
                elif any("Area_Chair" in s for s in sigs):
                    speaker = "Area Chair"
                text = node.content.get("comment", {}).get("value", "") or node.content.get("review", {}).get("value", "")
                transcript_lines.append(f"[{speaker}]: {text}")

            discussion_transcript = "\n\n".join(transcript_lines) if transcript_lines else "None"

            all_reviews_data.append({
                "paper_number": paper_number,
                "paper_title": paper_title,
                "decision": decision_value,
                "review_id": review_id,
                "rating": review_content.get("rating", {}).get("value"),
                "confidence": review_content.get("confidence", {}).get("value"),
                "review_text": review_content.get("review", {}).get("value"),
                "discussion_transcript": discussion_transcript,
            })

    return pd.DataFrame(all_reviews_data)

In [ ]:
df = get_iclr_2025_full_conversation()

print("\n---------------- RESULTS ----------------")
if not df.empty:
    print(f"Collected {len(df)} reviews with full conversation threads.")
    mask = df["discussion_transcript"].astype(str).str.contains("Reviewer", na=False)
    if mask.any():
        print("\n--- Sample conversation ---")
        print(df.loc[mask, "discussion_transcript"].iloc[0][:500] + "...\n")
else:
    print("No data found.")

In [ ]:
out_path = OUTPUT_DIR / "iclr2025_full_transcript.csv"
df.to_csv(out_path, index=False)
print(f"Saved to {out_path}")